In [1]:
/* 27-5 */
USE msdb;
GO

DECLARE @DBName VARCHAR(128) = 'AdventureWorks2014'

SELECT d.name,ca.backup_finish_date AS LastFullBackup
		,bs.BACKUP_finish_date AS LastLogBackup
	FROM sys.databases d
		LEFT OUTER JOIN msdb.dbo.backupset bs
			ON d.name = bs.database_name
			AND bs.TYPE = 'l'
		OUTER APPLY (SELECT TOP 1 database_name,backup_finish_date
						FROM msdb.dbo.backupset
						WHERE database_name = d.name
							AND type = 'D'
						ORDER BY backup_finish_date DESC) ca
	WHERE d.recovery_model_desc <> 'simple'
		AND bs.database_name IS NULL
		AND d.name = @DBName;
GO

Commands completed successfully.

(0 rows affected)

Total execution time: 00:00:00.259

name,LastFullBackup,LastLogBackup


In [2]:
/* whoops we have no log backup so take a log backup */
USE master;
GO

BACKUP LOG AdventureWorks2014
TO DISK = 'C:\Apress\AdventureWorks2014.trn';

Commands completed successfully.

: Msg 4208, Level 16, State 1, Line 5
The statement BACKUP LOG is not allowed while the recovery model is SIMPLE. Use BACKUP DATABASE or change the recovery model using ALTER DATABASE.

: Msg 3013, Level 16, State 1, Line 5
BACKUP LOG is terminating abnormally.

Total execution time: 00:00:00.025

In [3]:
/* some additional checks */
USE master;
GO

CREATE DATABASE Logging;
GO

Commands completed successfully.

: Msg 1801, Level 16, State 3, Line 5
Database 'Logging' already exists. Choose a different database name.

Total execution time: 00:00:00.004

In [4]:
ALTER DATABASE Logging
SET RECOVERY SIMPLE;
GO

USE Logging;
GO

CREATE TABLE FillErUp(
RowInfo	CHAR(150)
);
GO

Commands completed successfully.

Commands completed successfully.

: Msg 2714, Level 16, State 6, Line 8
There is already an object named 'FillErUp' in the database.

Total execution time: 00:00:01.312

In [5]:
/* check catalog views */
USE master;
GO
DECLARE @DBName VARCHAR(128) = 'Logging';

SELECT d.name as DBName, mf.size, d.recovery_model_desc, d.log_reuse_wait_desc
	FROM sys.master_files mf
		INNER JOIN sys.databases d
			ON d.database_id = mf.database_id
	WHERE d.name = @DBName
		AND mf.type_desc = 'LOG';
GO

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.173

DBName,size,recovery_model_desc,log_reuse_wait_desc
Logging,9216,SIMPLE,NOTHING


In [6]:
USE Logging;
GO
SET NOCOUNT ON;
DECLARE @count INT = 10000
WHILE @count > 0
BEGIN
   INSERT INTO FillErUp (RowInfo)
   SELECT 'This is row # ' + CONVERT(CHAR(4), @count)
   SET @count -= 1
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:02.955

In [7]:
CHECKPOINT;
GO

Commands completed successfully.

Total execution time: 00:00:00.018

In [8]:
/* check catalog views */
USE master;
GO
DECLARE @DBName VARCHAR(128) = 'Logging';

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.006

In [9]:
SELECT d.name as DBName, mf.size, d.recovery_model_desc, d.log_reuse_wait_desc
	FROM sys.master_files mf
		INNER JOIN sys.databases d
			ON d.database_id = mf.database_id
	WHERE d.name = @DBName
		AND mf.type_desc = 'LOG';

: Msg 137, Level 15, State 2, Line 5
Must declare the scalar variable "@DBName".

Total execution time: 00:00:00.001

In [10]:
/* long running transaction */
USE Logging;
GO

BEGIN TRAN

DECLARE @count INT = 10000
WHILE @count > 0
BEGIN
   INSERT INTO FillErUp
   SELECT 'This is row # ' + CONVERT(CHAR(4), @count)
   SET @count -= 1
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.269

In [11]:
USE master;
GO
DECLARE @DBName VARCHAR(128) = 'Logging';

SELECT d.name as DBName, mf.size, d.recovery_model_desc, d.log_reuse_wait_desc
	FROM sys.master_files mf
		INNER JOIN sys.databases d
			ON d.database_id = mf.database_id
	WHERE d.name = @DBName
		AND mf.type_desc = 'LOG';

DBCC SQLPERF(LOGSPACE);
GO

Commands completed successfully.

DBCC execution completed. If DBCC printed error messages, contact your system administrator.

Total execution time: 00:00:00.124

DBName,size,recovery_model_desc,log_reuse_wait_desc
Logging,9216,SIMPLE,NOTHING


Database Name,Log Size (MB),Log Space Used (%),Status
master,1.9921875,35.122547,0
tempdb,7.9921875,7.673509,0
model,7.9921875,7.7040567,0
msdb,21.617188,6.5684857,0
AdventureWorks2014,17.992188,46.868214,0
Errors,7.9921875,10.111193,0
InMemory,7.9921875,10.331134,0
TSQLRecipe_A,7.9921875,10.685484,0
TestDB,7.9921875,9.365836,0
Logging,71.99219,28.548563,0


In [12]:
USE Logging;
GO

COMMIT TRAN;
GO
CHECKPOINT;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.014

In [13]:
USE master;
GO
DECLARE @DBName VARCHAR(128) = 'Logging';

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.004

In [14]:
SELECT d.name as DBName, mf.size, d.recovery_model_desc, d.log_reuse_wait_desc
	FROM sys.master_files mf
		INNER JOIN sys.databases d
			ON d.database_id = mf.database_id
	WHERE d.name = @DBName
		AND mf.type_desc = 'LOG';

DBCC SQLPERF(LOGSPACE);
GO

: Msg 137, Level 15, State 2, Line 5
Must declare the scalar variable "@DBName".

Total execution time: 00:00:00.005

In [15]:
/* full recovery */
USE master;
GO

--Create the Logging database
IF EXISTS (SELECT * FROM sys.databases WHERE name = 'Logging')
BEGIN
DROP DATABASE Logging;
END
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.120

In [16]:
CREATE DATABASE Logging
ON PRIMARY
( NAME = N'Logging', FILENAME = N'C:\APRESS\Logging.mdf' , SIZE = 4096KB  )
 LOG ON 
( NAME = N'Logging_log', FILENAME = N'C:\APRESS\Logging_log.ldf' , SIZE = 512KB );
GO

: Msg 5133, Level 16, State 1, Line 1
Directory lookup for the file "C:\APRESS\Logging.mdf" failed with the operating system error 2(The system cannot find the file specified.).

: Msg 1802, Level 16, State 1, Line 1
CREATE DATABASE failed. Some file names listed could not be created. Check related errors.

Total execution time: 00:00:00.009

In [17]:
ALTER DATABASE Logging
SET RECOVERY FULL;
GO

: Msg 5011, Level 14, State 5, Line 1
User does not have permission to alter database 'Logging', the database does not exist, or the database is not in a state that allows access checks.

: Msg 5069, Level 16, State 1, Line 1
ALTER DATABASE statement failed.

Total execution time: 00:00:00.008

In [18]:
ALTER DATABASE Logging 
MODIFY FILE
    (NAME = Logging_log,
    SIZE = 520KB);
GO

: Msg 911, Level 16, State 1, Line 1
Database 'Logging' does not exist. Make sure that the name is entered correctly.

Total execution time: 00:00:00.005

In [19]:
--Size is 101
USE master;
GO
DECLARE @DBName VARCHAR(128) = 'Logging';

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.004

In [20]:
SELECT d.name as DBName, mf.size, d.recovery_model_desc, d.log_reuse_wait_desc
	FROM sys.master_files mf
		INNER JOIN sys.databases d
			ON d.database_id = mf.database_id
	WHERE d.name = @DBName
		AND mf.type_desc = 'LOG';
GO

: Msg 137, Level 15, State 2, Line 5
Must declare the scalar variable "@DBName".

Total execution time: 00:00:00.002

In [21]:
USE Logging;
GO
SELECT TOP 10000
        RowInfo = 'This is row # ' + CONVERT(CHAR(5)
		    , ROW_NUMBER() OVER (PARTITION BY t1.autoval ORDER BY (SELECT NULL)))
   INTO dbo.FillErUp
   FROM master.dbo.syscolumns t1,
        master.dbo.syscolumns t2;
GO

CHECKPOINT;
GO

: Msg 911, Level 16, State 1, Line 1
Database 'Logging' does not exist. Make sure that the name is entered correctly.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.221

In [22]:
USE master;
GO
DECLARE @DBName VARCHAR(128) = 'Logging';

SELECT d.name as DBName, mf.size, d.recovery_model_desc, d.log_reuse_wait_desc
	FROM sys.master_files mf
		INNER JOIN sys.databases d
			ON d.database_id = mf.database_id
	WHERE d.name = @DBName
		AND mf.type_desc = 'LOG';
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.010

DBName,size,recovery_model_desc,log_reuse_wait_desc


In [23]:
USE master;
GO

BACKUP DATABASE Logging
TO DISK =  'C:\Apress\Logging.bak';
GO

Commands completed successfully.

: Msg 911, Level 16, State 11, Line 4
Database 'Logging' does not exist. Make sure that the name is entered correctly.

: Msg 3013, Level 16, State 1, Line 4
BACKUP DATABASE is terminating abnormally.

Total execution time: 00:00:00.008

In [24]:
USE Logging;
GO
INSERT INTO dbo.FillErUp
SELECT TOP 10000
        RowInfo = 'This is row # ' + CONVERT(CHAR(5)
		    , ROW_NUMBER() OVER (PARTITION BY t1.autoval ORDER BY (SELECT NULL)))
   FROM master.dbo.syscolumns t1,
        master.dbo.syscolumns t2;
GO

: Msg 911, Level 16, State 1, Line 1
Database 'Logging' does not exist. Make sure that the name is entered correctly.

Commands completed successfully.

Total execution time: 00:00:00.235

In [25]:
CHECKPOINT;
GO

Commands completed successfully.

Total execution time: 00:00:00.010

In [26]:
USE master;
GO
DECLARE @DBName VARCHAR(128) = 'Logging';

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.001

In [27]:
SELECT d.name as DBName, mf.size, d.recovery_model_desc, d.log_reuse_wait_desc
	FROM sys.master_files mf
		INNER JOIN sys.databases d
			ON d.database_id = mf.database_id
	WHERE d.name = @DBName
		AND mf.type_desc = 'LOG';
GO

: Msg 137, Level 15, State 2, Line 5
Must declare the scalar variable "@DBName".

Total execution time: 00:00:00.002

In [28]:
/* bulk logged */

USE master;
GO

IF EXISTS (SELECT * FROM sys.databases WHERE name = 'Logging')
BEGIN
DROP DATABASE Logging;
END
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.024

In [29]:
CREATE DATABASE Logging;
GO

Commands completed successfully.

Total execution time: 00:00:00.291

In [30]:
ALTER DATABASE Logging
SET RECOVERY BULK_LOGGED;
GO

Commands completed successfully.

Total execution time: 00:00:00.008

In [31]:
BACKUP DATABASE Logging
TO DISK =  'C:\Apress\Logging_bulk.bak';
GO

: Msg 3201, Level 16, State 1, Line 1
Cannot open backup device 'C:\Apress\Logging_bulk.bak'. Operating system error 3(The system cannot find the path specified.).

: Msg 3013, Level 16, State 1, Line 1
BACKUP DATABASE is terminating abnormally.

Total execution time: 00:00:00.041

In [32]:
USE master;
GO
DECLARE @DBName VARCHAR(128) = 'Logging';

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.005

In [33]:
SELECT d.name as DBName, mf.size, d.recovery_model_desc, d.log_reuse_wait_desc
	FROM sys.master_files mf
		INNER JOIN sys.databases d
			ON d.database_id = mf.database_id
	WHERE d.name = @DBName
		AND mf.type_desc = 'LOG';
GO

: Msg 137, Level 15, State 2, Line 5
Must declare the scalar variable "@DBName".

Total execution time: 00:00:00

In [34]:
USE Logging;
GO

SELECT *
	INTO PurchaseOrderDetail
	FROM AdventureWorks2014.Purchasing.PurchaseOrderDetail

USE master;
GO
DECLARE @DBName VARCHAR(128) = 'Logging';

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.114

In [35]:
SELECT d.name as DBName, mf.size, d.recovery_model_desc, d.log_reuse_wait_desc
	FROM sys.master_files mf
		INNER JOIN sys.databases d
			ON d.database_id = mf.database_id
	WHERE d.name = @DBName
		AND mf.type_desc = 'LOG';
GO

: Msg 137, Level 15, State 2, Line 5
Must declare the scalar variable "@DBName".

Total execution time: 00:00:00.001